# Avance 2 – Modelo Relacional
### En este notebook se construye un modelo relacional a partir del dataset ventasTransformed.csv, normalizando la información en tablas de hechos y dimensiones, validando integridad referencial y generando un archivo Excel (modeloVentas.xlsx) que será utilizado como fuente del modelo en Power BI para el Avance 3.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("ventasTransformed.csv", encoding="latin1", sep=";")
df.head()

VentaID  FechaVenta HoraVenta        NombreSucursal CiudadSucursal  \
0      113  15/12/2017  10:02:23  Techcore Medellín #1       Medellín   
1      243    5/3/2024  11:04:28  Techcore Medellín #1       Medellín   
2      581   17/9/2023  06:57:54  Techcore Medellín #1       Medellín   
3      594   9/12/2014  00:07:40  Techcore Medellín #1       Medellín   
4      599    4/5/2023  01:00:16  Techcore Medellín #1       Medellín   

           NombreVendedor                    NombreCliente GeneroCliente  \
0  Liliana Jordá Armengol                  Alicia Del Gras             F   
1  Liliana Jordá Armengol            Paco Becerra Calderon             M   
2  Liliana Jordá Armengol  Pascual Erasmo Pastor Izquierdo             M   
3  Liliana Jordá Armengol            Francisco Vega-Alcalá             M   
4  Liliana Jordá Armengol             Elvira Rocamora Díaz             F   

   EdadCliente           EmailCliente  ...  NombreProducto3   MarcaProducto3  \
0           33     alicia52@yahoo.com  ...  No especificado  No especificado   
1           20       paco79@yahoo.com  ...  No especificado  No especificado   
2           29    pascual28@gmail.com  ...  No especificado  No especificado   
3           48  francisco50@yahoo.com  ...  No especificado  No especificado   
4           39   elvira59@hotmail.com  ...  No especificado  No especificado   

  CantidadProducto3 PrecioUnitarioProducto3 SubtotalProducto3  DescuentoVenta  \
0                 0                       0                 0               0   
1                 0                       0                 0               0   
2                 0                       0                 0               0   
3                 0                       0                 0               0   
4                 0                       0                 0               0   

   TotalVenta  AñoVenta MesVenta DiaVenta  
0    10400000      2017       12       15  
1     4400000      2024        3        5  
2     6800000      2023        9       17  
3     4000000      2014       12        9  
4     6000000      2023        5        4  

[5 rows x 33 columns]

In [3]:
df.columns

Index(['VentaID', 'FechaVenta', 'HoraVenta', 'NombreSucursal',
       'CiudadSucursal', 'NombreVendedor', 'NombreCliente', 'GeneroCliente',
       'EdadCliente', 'EmailCliente', 'TelefonoCliente', 'DireccionCliente',
       'MetodoPago', 'NombreProducto1', 'MarcaProducto1', 'CantidadProducto1',
       'PrecioUnitarioProducto1', 'SubtotalProducto1', 'NombreProducto2',
       'MarcaProducto2', 'CantidadProducto2', 'PrecioUnitarioProducto2',
       'SubtotalProducto2', 'NombreProducto3', 'MarcaProducto3',
       'CantidadProducto3', 'PrecioUnitarioProducto3', 'SubtotalProducto3',
       'DescuentoVenta', 'TotalVenta', 'AñoVenta', 'MesVenta', 'DiaVenta'],
      dtype='object')

### Creación de tablas dimensionales

In [4]:
ciudades = (
    df[['CiudadSucursal']]
    .drop_duplicates()
    .reset_index(drop=True)
)

ciudades['CiudadID'] = ciudades.index + 1
ciudades.rename(columns={'CiudadSucursal': 'NombreCiudad'}, inplace=True)

ciudades = ciudades[['CiudadID', 'NombreCiudad']]

Sucursales

In [5]:
sucursales_raw = df[['NombreSucursal', 'CiudadSucursal']]

sucursales = (
    sucursales_raw
    .value_counts()
    .reset_index(name='freq')
    .sort_values('freq', ascending=False)
    .drop_duplicates(subset=['NombreSucursal'])
)

sucursales = sucursales.merge(
    ciudades,
    left_on='CiudadSucursal',
    right_on='NombreCiudad',
    how='left'
)

sucursales['SucursalID'] = range(1, len(sucursales) + 1)

sucursales = sucursales[['SucursalID', 'NombreSucursal', 'CiudadID']]

Vendedores

In [6]:
vendedores = (
    df[['NombreVendedor', 'NombreSucursal']]
    .drop_duplicates()
    .merge(sucursales, on='NombreSucursal', how='left')
)

vendedores['VendedorID'] = range(1, len(vendedores) + 1)

vendedores = vendedores[['VendedorID', 'NombreVendedor', 'SucursalID']]

Clientes

In [7]:
clientes = (
    df[['NombreCliente', 'GeneroCliente', 'EdadCliente',
        'EmailCliente', 'TelefonoCliente', 'DireccionCliente']]
    .drop_duplicates()
    .reset_index(drop=True)
)

clientes['ClienteID'] = clientes.index + 1

clientes = clientes[['ClienteID', 'NombreCliente', 'GeneroCliente',
                     'EdadCliente', 'EmailCliente', 'TelefonoCliente',
                     'DireccionCliente']]

Productos

In [8]:
productos_list = []

for i in [1, 2, 3]:
    temp = df[[f'NombreProducto{i}', f'MarcaProducto{i}', f'PrecioUnitarioProducto{i}']]
    temp.columns = ['NombreProducto', 'Marca', 'PrecioUnitario']
    productos_list.append(temp)

productos = pd.concat(productos_list)
productos = productos[productos['NombreProducto'] != 'No especificado']
productos = productos.drop_duplicates().reset_index(drop=True)

productos['ProductoID'] = productos.index + 1
productos = productos[['ProductoID', 'NombreProducto', 'Marca', 'PrecioUnitario']]

Facturas

In [9]:
facturas = (
    df
    .merge(
        clientes,
        on=[
            'NombreCliente', 'GeneroCliente', 'EdadCliente',
            'EmailCliente', 'TelefonoCliente', 'DireccionCliente'
        ],
        how='left'
    )
    .merge(
        vendedores,
        on='NombreVendedor',
        how='left'
    )
)

facturas = facturas[[
    'VentaID', 'FechaVenta',
    'SucursalID', 'ClienteID', 'VendedorID',
    'MetodoPago', 'DescuentoVenta', 'TotalVenta'
]]

facturas.rename(columns={'VentaID': 'FacturaID'}, inplace=True)
facturas = facturas.drop_duplicates()

In [10]:
facturas.columns

Index(['FacturaID', 'FechaVenta', 'SucursalID', 'ClienteID', 'VendedorID',
       'MetodoPago', 'DescuentoVenta', 'TotalVenta'],
      dtype='object')

DetalleFacturas

In [11]:
detalles = []

for i in [1, 2, 3]:
    temp = df[[
        'VentaID',
        f'NombreProducto{i}', f'MarcaProducto{i}',
        f'CantidadProducto{i}', f'PrecioUnitarioProducto{i}',
        f'SubtotalProducto{i}'
    ]]

    temp.columns = [
        'FacturaID', 'NombreProducto', 'Marca',
        'Cantidad', 'PrecioUnitario', 'Subtotal'
    ]

    temp = temp[temp['NombreProducto'] != 'No especificado']
    detalles.append(temp)

detalle_facturas = pd.concat(detalles)
detalle_facturas = detalle_facturas.merge(
    productos,
    on=['NombreProducto', 'Marca', 'PrecioUnitario'],
    how='left'
)

detalle_facturas['DetalleID'] = range(1, len(detalle_facturas) + 1)

detalle_facturas = detalle_facturas[[
    'DetalleID', 'FacturaID', 'ProductoID',
    'Cantidad', 'PrecioUnitario', 'Subtotal'
]]

In [12]:
detalle_facturas['Cantidad'].value_counts()

Cantidad
1    30106
2    29953
Name: count, dtype: int64

### Validación de integridad referencial

In [13]:
assert detalle_facturas['FacturaID'].isin(facturas['FacturaID']).all()
assert detalle_facturas['ProductoID'].isin(productos['ProductoID']).all()

### Reportes rápidos de verificación

Total de ventas por marca 

In [14]:
ventas_por_marca = (
    detalle_facturas
    .merge(productos, on='ProductoID')
    .groupby('Marca')['Subtotal']
    .sum()
    .sort_values(ascending=False)
)

ventas_por_marca

Marca
Lenovo       656165600000
HP           529810000000
Dell         476146600000
Apple        422534400000
Asus         182941400000
Acer         121788800000
Samsung       67596000000
MSI           47754400000
Microsoft     45193600000
Razer         22274800000
Name: Subtotal, dtype: int64

Top 10 productos más vendidos

In [15]:
top_productos = (
    detalle_facturas
    .merge(productos, on='ProductoID')
    .groupby('NombreProducto')['Cantidad']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

top_productos

NombreProducto
Lenovo Legion 5 Pro          5861
Dell Latitude 7420           4507
Hp Spectre X360              3990
HP Spectre x360              3736
Dell Inspiron 15             3498
Lenovo ThinkPad X1 Carbon    2990
Lenovo Thinkpad X1 Carbon    2824
Lenovo Yoga 7i               2617
HP Omen 16                   2530
Lenovo IdeaPad 5             2509
Name: Cantidad, dtype: int64

### Tabla Usuarios

In [16]:
usuarios = pd.DataFrame({
    'UsuarioID': [1, 2, 3],
    'NombreUsuario': ['Gerente Nacional', 'Gerente Medellín', 'Gerente Techcore #1'],
    'EmailUsuario': ['nacional@empresa.com', 'regional@empresa.com', 'sucursal@empresa.com'],
    'Rol': ['Gerente Nacional', 'Gerente Regional', 'Gerente Sucursal'],
    'CiudadID': [np.nan, ciudades.loc[ciudades['NombreCiudad'] == 'Medellín', 'CiudadID'].values[0], np.nan],
    'SucursalID': [np.nan, np.nan, sucursales.loc[sucursales['NombreSucursal'] == 'Techcore Medellín #1', 'SucursalID'].values[0]]
})

## Exportación

In [17]:
with pd.ExcelWriter("modeloVentas.xlsx") as writer:
    facturas.to_excel(writer, sheet_name='Facturas', index=False)
    detalle_facturas.to_excel(writer, sheet_name='DetalleFacturas', index=False)
    productos.to_excel(writer, sheet_name='Productos', index=False)
    clientes.to_excel(writer, sheet_name='Clientes', index=False)
    sucursales.to_excel(writer, sheet_name='Sucursales', index=False)
    ciudades.to_excel(writer, sheet_name='Ciudades', index=False)
    vendedores.to_excel(writer, sheet_name='Vendedores', index=False)
    usuarios.to_excel(writer, sheet_name='Usuarios', index=False)